<a href="https://colab.research.google.com/github/hananbahtiti/Hybrid-Intrusion-detection-Systems/blob/main/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#test

In [ ]:
!wget -O NGIDS-DS.rar https://unsworks.unsw.edu.au/bitstreams/0ac2820a-5131-43ab-90b2-c624c8d73649/download

--2025-04-21 11:04:15--  https://unsworks.unsw.edu.au/bitstreams/0ac2820a-5131-43ab-90b2-c624c8d73649/download
Resolving unsworks.unsw.edu.au (unsworks.unsw.edu.au)... 54.253.215.118
Connecting to unsworks.unsw.edu.au (unsworks.unsw.edu.au)|54.253.215.118|:443... failed: Connection timed out.
Retrying.

--2025-04-21 11:06:28--  (try: 2)  https://unsworks.unsw.edu.au/bitstreams/0ac2820a-5131-43ab-90b2-c624c8d73649/download
Connecting to unsworks.unsw.edu.au (unsworks.unsw.edu.au)|54.253.215.118|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://unsworks.unsw.edu.au/server/api/core/bitstreams/0ac2820a-5131-43ab-90b2-c624c8d73649/content [following]
--2025-04-21 11:07:24--  https://unsworks.unsw.edu.au/server/api/core/bitstreams/0ac2820a-5131-43ab-90b2-c624c8d73649/content
Reusing existing connection to unsworks.unsw.edu.au:443.
HTTP request sent, awaiting response... No data received.
Retrying.

--2025-04-21 11:07:25--  (try: 2)  https://unsworks.unsw.

In [ ]:
!unrar x /content/NGIDS-DS.rar


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/NGIDS-DS.rar

Creating    NGIDS                                                     OK
Creating    NGIDS/NGIDS-DS-v1                                         OK
Extracting  NGIDS/NGIDS-DS-v1/feature_descr.csv                            0%  OK 
Extracting  NGIDS/NGIDS-DS-v1/ground_truth.csv                             0%  OK 
Creating    NGIDS/NGIDS-DS-v1/host logs                               OK
Extracting  NGIDS/NGIDS-DS-v1/host logs/1.csv                              0%  OK 
Extracting  NGIDS/NGIDS-DS-v1/host logs/10.csv                             0%  1%  OK 
Extracting  NGIDS/NGIDS-DS-v1/host logs/11.csv                             1%  OK 
Extracting  NGIDS/NGIDS-DS-v1/host logs/12.csv                             1%  2%  OK 
Extracting  NGIDS/NGIDS-DS-v1/host logs/13.csv                             2%  OK 
Extrac

In [ ]:
!rm -rf /content/NGIDS

#class

In [ ]:
!pip install tqdm

In [ ]:
import pandas as pd
import numpy as np
import os
import glob
import math
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

In [ ]:
class HostPreprocessing():
  def __init__(self):
    pass

  def __file_path_collection(self, folder_path):
    try:
      joined_list = []
      for file_name in os.listdir(folder_path):
        file_name = os.path.join(folder_path, file_name)
        if os.path.isfile(file_name) and file_name.endswith('.csv'):
          joined_list.append(file_name)
      return joined_list

    except Exception as e:
      return f"An error occurred: {e}"


  def rename_columns(self, folder_path):
    files = self.__file_path_collection(folder_path)
    try:
      for file in tqdm(files, desc="Renaming columns") :
        df = pd.read_csv(file)
        column_name = df.columns.tolist()
        df.rename(columns={
            column_name[0]:'Date',
            column_name[1]:'Time',
            column_name[2]:'Unique_Identification',
            column_name[3]:'Execution_Path',
            column_name[4]:'System_Calls_Identifiers',
            column_name[5]:"Event's_Unique_Identification",
            column_name[6]:'attacks',
            column_name[7]:"Sub_Type_Attack",
            column_name[8]:"Label"
            }, inplace=True)
        df.to_csv(file, index=False)
      return f"done files..."

    except Exception as e:
      return f"An error occurred: {e}"


  def data_encoding(self, folder_path, column_number : int):
    files = self.__file_path_collection(folder_path)
    encoder = LabelEncoder()
    all_value = []
    try:
      for file in tqdm(files, desc="Extract all values from files"):
        df = pd.read_csv(file)
        all_value.extend(df.iloc[:, column_number].dropna().unique())
        #print( len(df.iloc[:, column_number].dropna().unique()) , file )
      encoder.fit(all_value)

      for file in tqdm(files, desc="encoder data"):
        df = pd.read_csv(file)
        df.iloc[:, column_number] = encoder.transform(df.iloc[:, column_number])
        #print(len(df.iloc[:, column_number].dropna().unique()), df.iloc[:, column_number].dropna().unique(), file )
        df.to_csv(file, index=False)
      return f"done file..."

    except Exception as e:
      return f"An error occurred: {e}"


  def paths_encoding(self, folder_path, column_number: int):
    files = self.__file_path_collection(folder_path)
    all_unique_value = []
    unique_value = []
    try:
      for file in tqdm(files, desc="read all files"):
        df = pd.read_csv(file)
        unique_value.extend(df.iloc[:,column_number].unique())
        #df["Tokenized"] = unique_value.apply(lambda p: p.strip("/").split("/"))
        for tokens in unique_value:
          all_unique_value.extend(tokens)

      print( len(all_unique_value), all_unique_value)

    except Exception as e:
      return f"An error occurred: {e}"


  def file_collection(self, folder_path):
    try:
      file_paths = self.__file_path_collection(folder_path)
      if file_paths:
        output_folder = os.path.join(os.getcwd(),'files')
        os.makedirs(output_folder ,exist_ok=True)
        output_file = os.path.join(output_folder,"final.csv")

        for file in file_paths:

          first = True
          chunk = pd.read_csv(file, chunksize=10000)
          for part in chunk:
            part.to_csv(output_file, mode='a', header=first, index=False)
            first = False
          print(f"done file: {file}")

    except Exception as e:
      print(f"An error occurred: {e}")


a = Preprocessing()
#a.file_path_collection("/content/NGIDS/NGIDS-DS-v1/host logs")
#a.file_collection("/content/NGIDS/NGIDS-DS-v1/host logs")
#a.rename_columns( '/content/NGIDS/NGIDS-DS-v1/host logs')
#a.data_encoding('/content/NGIDS/NGIDS-DS-v1/host logs', column_number=6)
#a.data_encoding('/content/NGIDS/NGIDS-DS-v1/host logs', column_number=7)

#a.paths_encoding('/content/NGIDS/NGIDS-DS-v1/host logs', column_number=3)

encoder data: 100%|██████████| 99/99 [10:03<00:00,  6.10s/it]


'done file...'

#network




In [6]:
pip install scapy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 24.7 MB/s eta 0:00:00


In [16]:
import csv
import json
import os
import re
from tqdm import tqdm
from scapy.all import PcapReader

In [ ]:
class NetworkPreprocessing:
  def __init__(self):
    pass


  def extract_tcp_option(self, option, key):
    for opt in option:
      if opt[0] == key:
        return opt[1]
    return None


  def delete_data(self, file_path):
    df = pd.read_csv(file_path)
    df = df.loc[:, df.nunique() > 1]
    df.to_csv(file_path, index=False)
    return f"Columns containing only one value are deleted..."

  def convert_pcap_csv(self, pcap_file, csv_file):
    try:
      print("Countig packages...")
      with PcapReader(pcap_file) as count_reader:
        total_packets = sum(1 for _ in count_reader)
        print(f"Total number of packages: {total_packets}")

      print("Start converting to csv file...")
      with PcapReader(pcap_file) as packets:
        with open(csv_file, mode='w', newline="") as file:
          writer = csv.writer(file)
          writer.writerow([
              "Packet Number", "Ethernet DST", "Ethernet SRC", "Ethernet Type",
              "IP Version", "IP IHL", "IP TOS", "IP Length", "IP ID", "IP Flags",
              "IP Fragmentation", "IP TTL", "IP Proto", "IP Checksum",
              "IP Src", "IP Dst", "TCP Src Port", "TCP Dst Port", "TCP Seq",
              "TCP Ack", "TCP Data Offset", "TCP Reserved", "TCP Flags",
              "TCP Window", "TCP Checksum", "TCP Urgent Pointer", "TCP Options MSS",
              "TCP Options WScale", "TCP Options NOP","Hexdump"
          ])

          for i, packet in enumerate(tqdm(packets,total=total_packets, desc="Packet processing")):
            if packet.haslayer('Ethernet'):
              eth_dst = packet['Ethernet'].dst
              eth_src = packet['Ethernet'].src
              eth_type = packet['Ethernet'].type
            else:
              eth_dst = eth_src = eth_type = None

            if packet.haslayer('IP'):
              ip_version = packet['IP'].version
              ip_ihl = packet['IP'].ihl
              ip_tos = packet['IP'].tos
              ip_len = packet['IP'].len
              ip_id = packet['IP'].id
              ip_flags = packet['IP'].flags
              ip_frag = packet['IP'].frag
              ip_ttl = packet['IP'].ttl
              ip_proto = packet['IP'].proto
              ip_chksum = packet['IP'].chksum
              ip_src = packet['IP'].src
              ip_dst = packet['IP'].dst
            else:
              ip_version = ip_ihl = ip_tos = ip_len = ip_id = ip_flags = ip_frag = ip_ttl = ip_proto = ip_chksum = ip_src = ip_dst = None

            if packet.haslayer('TCP'):
              tcp_sport = packet['TCP'].sport
              tcp_dport = packet['TCP'].dport
              tcp_seq = packet['TCP'].seq
              tcp_ack = packet['TCP'].ack
              tcp_dataofs = packet['TCP'].dataofs
              tcp_reserved = packet['TCP'].reserved
              tcp_flags = packet['TCP'].flags
              tcp_window = packet['TCP'].window
              tcp_chksum = packet['TCP'].chksum
              tcp_urgptr = packet['TCP'].urgptr
              tcp_options = packet['TCP'].options

              tcp_mss = self.extract_tcp_option(tcp_options, 'MSS')
              tcp_wscale = self.extract_tcp_option(tcp_options, 'WScale')
              tcp_nop_count = self.extract_tcp_option(tcp_options, 'NOP')

            else:
              tcp_sport = tcp_dport = tcp_seq = tcp_ack = tcp_dataofs = tcp_reserved = tcp_flags = tcp_window = tcp_chksum = tcp_urgptr = tcp_mss = tcp_wscale = tcp_nop_count = None

            raw_data = packet.original.hex()
            writer.writerow([
                i + 1, eth_dst, eth_src, eth_type,
                  ip_version, ip_ihl, ip_tos, ip_len, ip_id, ip_flags,
                  ip_frag, ip_ttl, ip_proto, ip_chksum,
                  ip_src, ip_dst, tcp_sport, tcp_dport, tcp_seq,
                  tcp_ack, tcp_dataofs, tcp_reserved, tcp_flags,
                  tcp_window, tcp_chksum, tcp_urgptr, tcp_mss, tcp_wscale, tcp_nop_count,
                  raw_data
            ])

      return f"The file has been converted to csv successfully."
    except Exception as e:
      return f"An error occurred: {e}"

a = NetworkPreprocessing()
#a.convert_pcap_csv(pcap_file='/content/NGIDS/NGIDS-DS-v1/NGIDS.pcap',
#                   csv_file='/content/drive/MyDrive/hybrid_IDS/network.csv')
#a.delete_data(file_path='/content/drive/MyDrive/hybrid_IDS/network.csv')

'Columns containing only one value are deleted...'

In [14]:
class PcapFileProcessing:
  def __init__(self) -> None:
    pass


  def convert_log_csv(self, log_file, csv_file):
    try:
      print("Start converting to csv file...")
      with open(log_file, "r") as logfile:
        log_data = logfile.read()
      pattern = re.compile(
          r'(?P<timestamp>\d{2}/\d{2}/\d{4}-\d{2}:\d{2}:\d{2}\.\d+)\s+\[\*\*\]\s+\[(?P<sid>[^]]+)\]\s+(?P<signature>.*?)\s+\[\*\*\]\s+\[Classification:\s+(?P<classification>.*?)\]\s+\[Priority:\s+(?P<priority>\d+)\]\s+\{(?P<protocol>\w+)\}\s+(?P<src_ip>\d+\.\d+\.\d+\.\d+):(?P<src_port>\d+)\s+->\s+(?P<dst_ip>\d+\.\d+\.\d+\.\d+):(?P<dst_port>\d+)'
          )
      with open(csv_file, "w", newline="") as csvfile:
        fieldnames = ['timestamp', 'sid', 'signature', 'classification',
                      'priority', 'protocol','src_ip', 'src_port', 'dst_ip', 'dst_port']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for match in pattern.finditer(log_data):
          writer.writerow(match.groupdict())

      return f"The file has been converted to csv successfully."

    except Exception as e:
      return f"An error occurred: {e}"


  def convert_json_csv(self, json_file, csv_file):
    try:
      print("Start converting JSON to CSV...")
      with open(json_file, 'r') as f:
        data = [json.loads(line) for line in f]

      with open(csv_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        fieldnames = set()
        for row in data:
          fieldnames.update(row.keys())
        writer.writerow(fieldnames)

        for row in data:
          writer.writerow(row)

      return f"The file has been converted to csv successfully."

    except Exception as e:
      return f"An error occurred: {e}"



a = PcapFileProcessing()
a.convert_log_csv( log_file="/content/drive/MyDrive/hybrid_IDS/fast.log",
                  csv_file="/content/drive/MyDrive/hybrid_IDS/alerts.csv")

Start converting to csv file...


'The file has been converted to csv successfully.'

In [ ]:
df.nunique()

,0
Packet Number,1094232
Ethernet DST,5
Ethernet SRC,2
Ethernet Type,3
IP IHL,6
IP Length,845
IP ID,65536
IP Fragmentation,2
IP TTL,3
IP Proto,7


In [ ]:
df.dropna(inplace=True)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1063669 entries, 0 to 1094220
Data columns (total 23 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   Packet Number     1063669 non-null  int64  
 1   Ethernet DST      1063669 non-null  object 
 2   Ethernet SRC      1063669 non-null  object 
 3   Ethernet Type     1063669 non-null  int64  
 4   IP IHL            1063669 non-null  float64
 5   IP Length         1063669 non-null  float64
 6   IP ID             1063669 non-null  float64
 7   IP Fragmentation  1063669 non-null  float64
 8   IP TTL            1063669 non-null  float64
 9   IP Proto          1063669 non-null  float64
 10  IP Checksum       1063669 non-null  float64
 11  IP Src            1063669 non-null  object 
 12  IP Dst            1063669 non-null  object 
 13  TCP Src Port      1063669 non-null  float64
 14  TCP Dst Port      1063669 non-null  float64
 15  TCP Seq           1063669 non-null  float64
 16  TCP A

,Packet Number,Ethernet DST,Ethernet SRC,Ethernet Type,IP IHL,IP Length,IP ID,IP Fragmentation,IP TTL,IP Proto,...,TCP Src Port,TCP Dst Port,TCP Seq,TCP Ack,TCP Data Offset,TCP Flags,TCP Window,TCP Checksum,TCP Options MSS,Hexdump
0,1,02:1a:c5:00:00:00,02:1a:c5:00:00:00,2048,5.0,56.0,21299.0,0.0,32.0,6.0,...,60294.0,80.0,4.048308e+09,0.0,9.0,S,5792.0,16020.0,1460.0,021ac5000000021ac50000000800450000385333400020...
1,2,02:1a:c5:00:00:00,02:1a:c5:00:00:00,2048,5.0,56.0,21281.0,0.0,32.0,6.0,...,22311.0,1024.0,4.048306e+09,0.0,9.0,S,5792.0,54925.0,1460.0,021ac5000000021ac50000000800450000385321400020...
2,3,02:1a:c5:00:00:00,02:1a:c5:00:00:00,2048,5.0,56.0,21467.0,0.0,32.0,6.0,...,63239.0,27322.0,4.100437e+09,0.0,9.0,S,5792.0,38924.0,1460.0,021ac5000000021ac500000008004500003853db400020...
3,4,02:1a:c5:00:00:00,02:1a:c5:00:00:00,2048,5.0,56.0,21295.0,0.0,32.0,6.0,...,37241.0,23962.0,4.160434e+09,0.0,9.0,S,5792.0,21918.0,1460.0,021ac5000000021ac5000000080045000038532f400020...
4,5,02:1a:c5:00:00:00,02:1a:c5:00:00:00,2048,5.0,56.0,21469.0,0.0,32.0,6.0,...,17808.0,80.0,2.070854e+09,0.0,9.0,S,5792.0,48977.0,1460.0,021ac5000000021ac500000008004500003853dd400020...


#test

In [ ]:
unique_attacks = list(set([item for sub in c for item in sub]))
unique_attacks

['Exploits',
 'Generic',
 'Shellcode',
 'Worms',
 'Reconnaissance',
 'Denial of Service',
 'Backdoors',
 'normal']

In [ ]:
pip install scapy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 21.6 MB/s eta 0:00:00


In [15]:
df = pd.read_csv("/content/drive/MyDrive/hybrid_IDS/alerts.csv")
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5100 entries, 0 to 5099
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   timestamp       5100 non-null   object
 1   sid             5100 non-null   object
 2   signature       5100 non-null   object
 3   classification  5100 non-null   object
 4   priority        5100 non-null   int64 
 5   protocol        5100 non-null   object
 6   src_ip          5100 non-null   object
 7   src_port        5100 non-null   int64 
 8   dst_ip          5100 non-null   object
 9   dst_port        5100 non-null   int64 
dtypes: int64(3), object(7)
memory usage: 398.6+ KB


,timestamp,sid,signature,classification,priority,protocol,src_ip,src_port,dst_ip,dst_port
0,03/10/2016-17:49:10.892710,1:2200035:2,SURICATA TCP invalid option length,Generic Protocol Command Decode,3,TCP,175.45.176.1,0,10.40.85.32,0
1,03/10/2016-17:49:44.321710,1:2200036:2,SURICATA TCP option invalid length,Generic Protocol Command Decode,3,TCP,175.45.176.3,1024,10.40.85.32,4096
2,03/10/2016-17:49:44.321710,1:2200074:2,SURICATA TCPv4 invalid checksum,Generic Protocol Command Decode,3,TCP,175.45.176.3,1024,10.40.85.32,4096
3,03/10/2016-17:52:34.179588,1:2200004:2,SURICATA IPv4 invalid option,Generic Protocol Command Decode,3,TCP,175.45.176.3,36645,10.40.85.32,30989
4,03/10/2016-17:52:34.179588,1:2200008:2,SURICATA IPv4 option end of list required,Generic Protocol Command Decode,3,TCP,175.45.176.3,36645,10.40.85.32,30989


In [ ]:
df['TCP Options'].value_counts()

,count
TCP Flags,
S,799771
R,264798
RA,961
FA,100


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/hybrid_IDS/network.csv")
#df['Sub_Type_Attack'].value_counts()
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1094232 entries, 0 to 1094231
Data columns (total 30 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   Packet Number       1094232 non-null  int64  
 1   Ethernet DST        1094232 non-null  object 
 2   Ethernet SRC        1094232 non-null  object 
 3   Ethernet Type       1094232 non-null  int64  
 4   IP Version          1079094 non-null  float64
 5   IP IHL              1079094 non-null  float64
 6   IP TOS              1079094 non-null  float64
 7   IP Length           1079094 non-null  float64
 8   IP ID               1079094 non-null  float64
 9   IP Flags            2274 non-null     object 
 10  IP Fragmentation    1079094 non-null  float64
 11  IP TTL              1079094 non-null  float64
 12  IP Proto            1079094 non-null  float64
 13  IP Checksum         1079094 non-null  float64
 14  IP Src              1079094 non-null  object 
 15  IP Dst         

,Packet Number,Ethernet DST,Ethernet SRC,Ethernet Type,IP Version,IP IHL,IP TOS,IP Length,IP ID,IP Flags,...,TCP Data Offset,TCP Reserved,TCP Flags,TCP Window,TCP Checksum,TCP Urgent Pointer,TCP Options MSS,TCP Options WScale,TCP Options NOP,Hexdump
0,1,02:1a:c5:00:00:00,02:1a:c5:00:00:00,2048,4.0,5.0,0.0,56.0,21299.0,DF,...,9.0,0.0,S,5792.0,16020.0,0.0,1460.0,NaN,NaN,021ac5000000021ac50000000800450000385333400020...
1,2,02:1a:c5:00:00:00,02:1a:c5:00:00:00,2048,4.0,5.0,0.0,56.0,21281.0,DF,...,9.0,0.0,S,5792.0,54925.0,0.0,1460.0,NaN,NaN,021ac5000000021ac50000000800450000385321400020...
2,3,02:1a:c5:00:00:00,02:1a:c5:00:00:00,2048,4.0,5.0,0.0,56.0,21467.0,DF,...,9.0,0.0,S,5792.0,38924.0,0.0,1460.0,NaN,NaN,021ac5000000021ac500000008004500003853db400020...
3,4,02:1a:c5:00:00:00,02:1a:c5:00:00:00,2048,4.0,5.0,0.0,56.0,21295.0,DF,...,9.0,0.0,S,5792.0,21918.0,0.0,1460.0,NaN,NaN,021ac5000000021ac5000000080045000038532f400020...
4,5,02:1a:c5:00:00:00,02:1a:c5:00:00:00,2048,4.0,5.0,0.0,56.0,21469.0,DF,...,9.0,0.0,S,5792.0,48977.0,0.0,1460.0,NaN,NaN,021ac5000000021ac500000008004500003853dd400020...


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/hybrid_IDS/network.csv")
#df['Sub_Type_Attack'].value_counts()
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1094232 entries, 0 to 1094231
Data columns (total 23 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   Packet Number     1094232 non-null  int64  
 1   Ethernet DST      1094232 non-null  object 
 2   Ethernet SRC      1094232 non-null  object 
 3   Ethernet Type     1094232 non-null  int64  
 4   IP IHL            1079094 non-null  float64
 5   IP Length         1079094 non-null  float64
 6   IP ID             1079094 non-null  float64
 7   IP Fragmentation  1079094 non-null  float64
 8   IP TTL            1079094 non-null  float64
 9   IP Proto          1079094 non-null  float64
 10  IP Checksum       1079094 non-null  float64
 11  IP Src            1079094 non-null  object 
 12  IP Dst            1079094 non-null  object 
 13  TCP Src Port      1065930 non-null  float64
 14  TCP Dst Port      1065930 non-null  float64
 15  TCP Seq           1065930 non-null  float64
 16  

,Packet Number,Ethernet DST,Ethernet SRC,Ethernet Type,IP IHL,IP Length,IP ID,IP Fragmentation,IP TTL,IP Proto,...,TCP Src Port,TCP Dst Port,TCP Seq,TCP Ack,TCP Data Offset,TCP Flags,TCP Window,TCP Checksum,TCP Options MSS,Hexdump
0,1,02:1a:c5:00:00:00,02:1a:c5:00:00:00,2048,5.0,56.0,21299.0,0.0,32.0,6.0,...,60294.0,80.0,4.048308e+09,0.0,9.0,S,5792.0,16020.0,1460.0,021ac5000000021ac50000000800450000385333400020...
1,2,02:1a:c5:00:00:00,02:1a:c5:00:00:00,2048,5.0,56.0,21281.0,0.0,32.0,6.0,...,22311.0,1024.0,4.048306e+09,0.0,9.0,S,5792.0,54925.0,1460.0,021ac5000000021ac50000000800450000385321400020...
2,3,02:1a:c5:00:00:00,02:1a:c5:00:00:00,2048,5.0,56.0,21467.0,0.0,32.0,6.0,...,63239.0,27322.0,4.100437e+09,0.0,9.0,S,5792.0,38924.0,1460.0,021ac5000000021ac500000008004500003853db400020...
3,4,02:1a:c5:00:00:00,02:1a:c5:00:00:00,2048,5.0,56.0,21295.0,0.0,32.0,6.0,...,37241.0,23962.0,4.160434e+09,0.0,9.0,S,5792.0,21918.0,1460.0,021ac5000000021ac5000000080045000038532f400020...
4,5,02:1a:c5:00:00:00,02:1a:c5:00:00:00,2048,5.0,56.0,21469.0,0.0,32.0,6.0,...,17808.0,80.0,2.070854e+09,0.0,9.0,S,5792.0,48977.0,1460.0,021ac5000000021ac500000008004500003853dd400020...
